In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-10-13T10:59:08.252422-04:00

Python implementation: CPython
Python version       : 3.8.9
IPython version      : 8.4.0

Compiler    : Clang 13.1.6 (clang-1316.0.21.2.5)
OS          : Darwin
Release     : 21.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import xgboost as xgb



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/.DS_Store
../input/test.zip
../input/train.zip
../input/sample_submission.csv


In [4]:
%watermark --iversions

pandas : 1.5.0
numpy  : 1.23.4
xgboost: 1.6.2



In [21]:
def MCRMSE(truth, pred): 
    rmse1 = mean_squared_error(truth[:,0], pred[:,0], squared=False)
    rmse2 = mean_squared_error(truth[:,1], pred[:,1], squared=False) 
    rmse3 = mean_squared_error(truth[:,2], pred[:,2], squared=False)
    rmse4 = mean_squared_error(truth[:,3], pred[:,3], squared=False) 
    rmse5 = mean_squared_error(truth[:,4], pred[:,4], squared=False)
    return (rmse1 + rmse2 + rmse3+rmse4+rmse5)/5

In [5]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
train = pd.read_csv('../input/train.zip')
test = pd.read_csv('../input/test.zip')

In [6]:
train.head()

,PIDN,m7497.96,m7496.04,m7494.11,m7492.18,m7490.25,m7488.32,m7486.39,m7484.46,m7482.54,...,REF7,RELI,TMAP,TMFI,Depth,Ca,P,pH,SOC,Sand
0,XNhoFZW5,0.302553,0.301137,0.299748,0.300354,0.302679,0.303799,0.301702,0.298936,0.298126,...,-0.646673,1.687734,0.190708,0.056843,Topsoil,-0.295749,-0.041336,-1.129366,0.353258,1.269748
1,9XNspFTd,0.270192,0.268555,0.266964,0.267938,0.271013,0.272346,0.269870,0.266976,0.266544,...,-0.646673,1.687734,0.190708,0.056843,Subsoil,-0.387442,-0.231552,-1.531538,-0.264023,1.692209
2,WDId41qG,0.317433,0.316265,0.314948,0.315224,0.316942,0.317764,0.316067,0.313874,0.313301,...,-0.814516,1.806660,0.190708,0.056843,Topsoil,-0.248601,-0.224635,-0.259551,0.064152,2.091835
3,JrrJf1mN,0.261116,0.259767,0.258384,0.259001,0.261310,0.262417,0.260534,0.258039,0.257246,...,-0.814516,1.806660,0.190708,0.056843,Subsoil,-0.332195,-0.318014,-0.577548,-0.318719,2.118477
4,ZoIitegA,0.260038,0.258425,0.256544,0.257030,0.259602,0.260786,0.258717,0.256352,0.255902,...,-0.780242,0.430513,0.190708,0.056843,Topsoil,-0.438350,-0.010210,-0.699135,-0.310905,2.164148


In [7]:
train.shape

(1157, 3600)

In [8]:
test.shape

(727, 3595)

In [9]:
test.head()

,PIDN,m7497.96,m7496.04,m7494.11,m7492.18,m7490.25,m7488.32,m7486.39,m7484.46,m7482.54,...,LSTD,LSTN,REF1,REF2,REF3,REF7,RELI,TMAP,TMFI,Depth
0,09gt9UK5,0.210899,0.209221,0.207645,0.208714,0.211972,0.213562,0.211267,0.208248,0.207341,...,-0.009394,0.197885,-0.622545,-0.548238,-0.637681,-0.413306,-0.130139,0.154013,-0.033905,Subsoil
1,0BVvxJ6a,0.191650,0.184120,0.177070,0.180624,0.192343,0.197502,0.187655,0.175447,0.172639,...,-0.277944,-0.851643,-0.792000,-1.058442,-0.792961,-0.785786,-0.588855,0.106200,0.559694,Subsoil
2,0KbdgApg,0.083966,0.074740,0.065596,0.069924,0.085145,0.092527,0.080625,0.064768,0.060597,...,-0.443809,-0.628839,-0.754909,-0.926716,-0.865424,-0.607359,-0.334013,-0.021042,0.182431,Topsoil
3,0MnuuduB,0.345962,0.338567,0.331786,0.336948,0.350820,0.356365,0.345285,0.332233,0.328589,...,-0.758853,-0.961080,-0.729455,-0.780148,-0.716356,-0.552923,-0.079171,0.394936,0.774477,Topsoil
4,0PcIsF9z,0.239019,0.232611,0.226266,0.229921,0.241821,0.247756,0.239242,0.227977,0.224762,...,-1.228289,1.156914,-0.696727,0.080705,0.118012,-0.920363,-0.266055,1.612625,1.199048,Subsoil


In [15]:
sample_submission.head()

,PIDN,Ca,P,pH,SOC,Sand
0,09gt9UK5,0,0,0,0,0
1,0BVvxJ6a,0,0,0,0,0
2,0KbdgApg,0,0,0,0,0
3,0MnuuduB,0,0,0,0,0
4,0PcIsF9z,0,0,0,0,0


In [10]:
columns = test.columns[1:-1]
columns

Index(['m7497.96', 'm7496.04', 'm7494.11', 'm7492.18', 'm7490.25', 'm7488.32',
       'm7486.39', 'm7484.46', 'm7482.54', 'm7480.61',
       ...
       'EVI', 'LSTD', 'LSTN', 'REF1', 'REF2', 'REF3', 'REF7', 'RELI', 'TMAP',
       'TMFI'],
      dtype='object', length=3593)

In [11]:
X = train[columns].values
X_test = test[columns].values
target = train[['Ca', 'P', 'pH', 'SOC', 'Sand']].values

In [12]:
target

array([[-0.29574924, -0.04133638, -1.12936595,  0.35325838,  1.26974781],
       [-0.38744188, -0.23155201, -1.53153828, -0.26402306,  1.69220923],
       [-0.24860129, -0.22463507, -0.25955137,  0.06415188,  2.09183489],
       ...,
       [-0.44644913, -0.38026604, -0.40919689,  4.36949458, -0.40487401],
       [-0.49301858, -0.41830917, -0.19408146,  2.53327764, -0.56853023],
       [-0.20897388, -0.17967502, -0.54948957,  3.97099643, -0.42009784]])

In [22]:
%%time
train_oof = np.zeros(target.shape)
test_preds = 0
kf = KFold(n_splits=5, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_x = X[train_index]
    val_x = X[val_index]
    train_target = target[train_index]
    val_target = target[val_index]
    model = xgb.XGBRegressor(tree_method='hist')
    model.fit(train_x, train_target)
    val_preds = model.predict(val_x)
    train_oof[val_index] = val_preds
    test_preds += model.predict(X_test)/5
    print(MCRMSE(val_target, val_preds))

Fitting fold 1
0.533362779191431
Fitting fold 2
0.6165388230994064
Fitting fold 3
0.44746915912467183
Fitting fold 4
0.5632444929415477
Fitting fold 5
0.5844327628781342
CPU times: user 21min 33s, sys: 13min 13s, total: 34min 47s
Wall time: 2min 26s


In [23]:
MCRMSE(target, train_oof)

0.5563222045487125

In [24]:
sample_submission[['Ca', 'P', 'pH', 'SOC', 'Sand']] = test_preds
sample_submission.head()

,PIDN,Ca,P,pH,SOC,Sand
0,09gt9UK5,-0.474595,-0.267078,-0.626614,-0.688458,0.990249
1,0BVvxJ6a,0.129742,0.366407,0.422675,0.375519,-0.923167
2,0KbdgApg,-0.093051,-0.342664,0.191573,0.025721,-0.532031
3,0MnuuduB,0.026055,-0.186125,-0.166213,1.418597,-1.176176
4,0PcIsF9z,-0.470689,-0.116390,-1.023802,0.300560,0.368996


In [25]:
sample_submission.to_csv('xgb_0.csv', index=False)

Public score: 0.65117

Private score: 0.66096